作用: 加速深层网络的收敛速度；更深层的网络很复杂，容易过拟合, 批量规范化起到正则化的作用.

中间层中的变量（例如，多层感知机中的仿射变换输出）可能具有更广的变化范围, 如果一个层的可变值是另一层的100倍，对这两层而言梯度和学习率的效果就不一样.


正是由于这个基于批量统计的标准化，才有了批量规范化的名称。


计算流程:
- 首先规范化输入

  即通过减去其均值并除以其标准差，其中两者均基于当前小批量处理。 
  
- 接下来，我们应用比例系数和比例偏移。 

应用标准化后，生成的小批量的平均值为0和单位方差为1。

$$\begin{aligned} 
\mathrm{BN}(\mathbf{x}) &= \boldsymbol{\gamma} \odot \frac{\mathbf{x} - \hat{\boldsymbol{\mu}}_\mathcal{B}}{\hat{\boldsymbol{\sigma}}_\mathcal{B}} + \boldsymbol{\beta}, \\
\hat{\boldsymbol{\mu}}_\mathcal{B} &= \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} \mathbf{x},\\
\hat{\boldsymbol{\sigma}}_\mathcal{B}^2 &= \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} (\mathbf{x} - \hat{\boldsymbol{\mu}}_{\mathcal{B}})^2 + \epsilon
\end{aligned}$$
- $\mathbf{x} \in \mathcal{B}$表示一个来自小批量$\mathcal{B}$的输入
- $\hat{\boldsymbol{\mu}}_\mathcal{B}$是小批量$\mathcal{B}$的样本均值，$\hat{\boldsymbol{\sigma}}_\mathcal{B}$是小批量$\mathcal{B}$的样本标准差。
- $\boldsymbol{\gamma}$和$\boldsymbol{\beta}$是需要与其他模型参数一起学习的参数。
- 我们在方差估计值中添加一个小的常量$\epsilon > 0$，以确保我们永远不会尝试除以零

如果我们尝试使用**大小为1的小批量**应用批量规范化，我们将无法学到任何东西。 这是因为在减去均值之后，每个隐藏单元将为0。 所以，只有使用足够大的小批量，批量规范化这种方法才是有效且稳定的。 请注意，在应用批量规范化时，批量大小的选择可能比没有批量规范化时更重要。

批量规范化最适应$50 \sim 100$范围中的中等批量大小。




批量规范化是在卷积层或全连接层之后、相应的激活函数之前应用的。

- 全连接层:

  $$\mathbf{h} = \phi(\mathrm{BN}(\mathbf{W}\mathbf{x} + \mathbf{b}) ).$$

- 卷积层:

  当卷积有多个输出通道时，对这些通道的“每个”输出执行批量规范化.
  
  假设我们的小批量包含$m$个样本，并且对于每个通道，卷积的输出具有高度$p$和宽度$q$. 在每个输出通道的$m \cdot p \cdot q$个元素上同时执行每个批量规范化。
  
  每个通道都有自己的拉伸（scale）和偏移（shift）参数. 

批量规范化层和暂退层一样，在训练模式和预测模式下计算不同。

- 在训练过程中，我们无法得知使用整个数据集来估计平均值和方差，所以只能根据每个小批次的平均值和方差不断训练模型。（通过小批量统计数据规范化）

- 在预测模式下，可以根据整个数据集精确计算批量规范化所需的平均值和方差。通过数据集统计规范化）

## coding

In [1]:
nn.BatchNorm2d(16),

nn.BatchNorm1d(120), 

NameError: name 'nn' is not defined

例子: [](../net-example/lenet-batchnorm.ipynb)

In [ ]:
# 样本数为1就报错, 可能是因为方差为0?
X = torch.rand(size=(2, 1, 28, 28), dtype=torch.float32).to(device)
for layer in net:
    X = layer(X)
    print(f'output shape: {layer.__class__.__name__: <15}{X.shape}')